## preprocessing data into training data

In [1]:
import re, os

### removing page headers

In [2]:
# using regex to remove words that have all capital letters
with open('temp1.txt', 'w') as f:
    with open('data.txt', 'r') as g:
        for line in g:
            line = re.sub(r'\b[A-Z]{2,}\b', '', line)
            f.write(line)


### removing page footers

In [3]:
with open('temp2.txt', 'w') as f:
    with open('temp1.txt', 'r') as g:
        for line in g:
            # remove all numbers that have new line before and after using regex
            line = re.sub(r'\d+\n', '', line)
            # remove any weird characters
            line = re.sub(r'[|•]', '', line)
            f.write(line)

### creating sentences

In [4]:
with open('temp3.txt', 'w') as f:
    with open('temp2.txt', 'r') as g:
        for line in g:
            # remove all new lines using regex
            line = re.sub(r'\n', ' ', line)
            f.write(line)

In [5]:
# brute forcey
alphabets= "([A-Za-z])"
digits = "([0-9])"
prefixes = "(Mr|St|Mrs|Ms|Dr|Prof|Capt|Cpt|Lt|Mt)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov|me|edu)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

with open('output.txt', 'w') as f:
    with open('temp3.txt', 'r') as g:
        for line in g:
            sentences = split_into_sentences(line)
            for sentence in sentences:
                f.write(sentence + '\n')

In [6]:
# elegant nltk
import nltk
from nltk import tokenize
nltk.download('punkt')

with open('cleaned_data.txt', 'w') as f:
    with open('temp3.txt', 'r') as g:
        for line in g:
            sentences = tokenize.sent_tokenize(line)
            for sentence in sentences:
                # deleting sentences that have less than 3 words
                if len(sentence.split()) > 3:
                    # removing extra white spaces
                    sentence = " ".join(sentence.split())
                    f.write(sentence + '\n')

[nltk_data] Downloading package punkt to /home/hexagon/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
# cleanup
os.remove('temp1.txt')
os.remove('temp2.txt')
os.remove('temp3.txt')